# Pragmatic Asset Allocation - Portfolio Construction

This notebook analyzes the portfolio construction process, including tranche rebalancing, position sizing, and risk management.

## Objectives:
- Validate tranche rebalancing logic
- Analyze position sizing and turnover
- Evaluate risk management effectiveness
- Test portfolio construction in different market regimes

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load configuration
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully")
print(f"Strategy: {config['strategy']['name']}")
print(f"Tranche system: {config['portfolio']['tranches']['count']} tranches")
print(f"Rebalancing: {config['portfolio']['rebalancing']['frequency']}")

## 1. Load Portfolio Data

In [ ]:
# Load portfolio construction data
try:
    from portfolio_construction import PragmaticAssetAllocationPortfolio
    from signal_generation import PragmaticAssetAllocationSignals
    from data_acquisition import PragmaticAssetAllocationData
    
    # Load data and signals
    data_acq = PragmaticAssetAllocationData()
    all_data = data_acq.load_cached_data()
    
    if all_data:
        # Generate signals
        signal_gen = PragmaticAssetAllocationSignals()
        signals_dict = signal_gen.generate_all_signals(all_data)
        
        # Run portfolio construction
        portfolio = PragmaticAssetAllocationPortfolio()
        portfolio_data = portfolio.run_portfolio_construction(signals_dict, all_data)
        
        print("Portfolio construction completed successfully")
        print(f"Portfolio data keys: {list(portfolio_data.keys())}")
    else:
        print("No data available. Run data acquisition first.")
        portfolio_data = {}
        
except ImportError as e:
    print(f"Could not import modules: {e}")
    portfolio_data = {}

## 2. Tranche System Analysis

In [ ]:
# Analyze tranche system
if 'tranche_positions' in portfolio_data:
    tranche_positions = portfolio_data['tranche_positions']
    print("=== TRANCHE SYSTEM ANALYSIS ===\n")
    
    # Tranche summary
    print(f"Number of tranches: {len(tranche_positions)}")
    
    for tranche_name, positions in tranche_positions.items():
        if not positions.empty:
            print(f"\n{tranche_name}:")
            print(f"  Date range: {positions.index.min()} to {positions.index.max()}")
            print(f"  Total positions: {len(positions)}")
            
            # Asset allocation summary
            asset_cols = [col for col in positions.columns if not col.startswith('Cash') and col != 'Total_Value']
            if asset_cols:
                avg_allocation = positions[asset_cols].mean()
                print(f"  Average allocation per asset:")
                for asset, alloc in avg_allocation.items():
                    if alloc > 0.001:  # Only show meaningful allocations
                        print(f"    {asset}: {alloc:.1%}")
            
            # Cash allocation
            cash_cols = [col for col in positions.columns if col.startswith('Cash')]
            if cash_cols:
                avg_cash = positions[cash_cols].mean().sum()
                print(f"  Average cash allocation: {avg_cash:.1%}")
        
        # Visualize tranche allocations over time
        if not positions.empty:
            fig, axes = plt.subplots(2, 1, figsize=(15, 10))
            
            # Asset allocations
            if asset_cols:
                positions[asset_cols].plot(ax=axes[0], linewidth=1, alpha=0.7)
                axes[0].set_title(f'{tranche_name} - Asset Allocations')
                axes[0].set_ylabel('Allocation %')
                axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                axes[0].grid(True, alpha=0.3)
            
            # Cash allocation
            if cash_cols:
                positions[cash_cols].sum(axis=1).plot(ax=axes[1], linewidth=1, color='green')
                axes[1].set_title(f'{tranche_name} - Cash Allocation')
                axes[1].set_ylabel('Cash %')
                axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
    
    # Tranche rebalancing analysis
    print("\nTranche Rebalancing Analysis:")
    
    # Calculate rebalancing frequency
    rebalance_dates = {}
    for tranche_name, positions in tranche_positions.items():
        if not positions.empty:
            # Find dates where positions change significantly
            position_changes = positions.diff().abs().sum(axis=1)
            significant_changes = position_changes > 0.01  # 1% change threshold
            rebalance_dates[tranche_name] = positions.index[significant_changes]
            
            print(f"  {tranche_name}: {len(rebalance_dates[tranche_name])} rebalancing events")
    
    # Expected vs actual rebalancing frequency
    expected_rebalances_per_year = 4  # Quarterly
    holding_period_years = config['portfolio']['tranches']['holding_period_months'] / 12
    
    print(f"\nExpected rebalancing frequency: {expected_rebalances_per_year} times per year")
    print(f"Tranche holding period: {holding_period_years:.1f} years")
    
else:
    print("Tranche positions not available")

## 3. Combined Portfolio Analysis

In [ ]:
# Analyze combined portfolio
if 'combined_portfolio' in portfolio_data:
    combined_portfolio = portfolio_data['combined_portfolio']
    print("=== COMBINED PORTFOLIO ANALYSIS ===\n")
    
    # Portfolio summary
    print(f"Combined portfolio date range: {combined_portfolio.index.min()} to {combined_portfolio.index.max()}")
    print(f"Total observations: {len(combined_portfolio)}")
    
    # Asset allocation analysis
    asset_cols = [col for col in combined_portfolio.columns if not col.startswith('Cash') and col != 'Total_Value']
    cash_cols = [col for col in combined_portfolio.columns if col.startswith('Cash')]
    
    if asset_cols:
        # Average allocation
        avg_allocation = combined_portfolio[asset_cols].mean()
        print("\nAverage Asset Allocation:")
        for asset, alloc in avg_allocation.sort_values(ascending=False).items():
            if alloc > 0.001:
                print(f"  {asset}: {alloc:.1%}")
        
        # Allocation volatility
        alloc_volatility = combined_portfolio[asset_cols].std()
        print("\nAsset Allocation Volatility:")
        for asset, vol in alloc_volatility.sort_values(ascending=False).items():
            if vol > 0.001:
                print(f"  {asset}: {vol:.1%}")
        
        # Visualize portfolio allocation over time
        fig, axes = plt.subplots(2, 1, figsize=(15, 12))
        
        # Asset allocations
        combined_portfolio[asset_cols].plot(ax=axes[0], linewidth=1, alpha=0.7)
        axes[0].set_title('Combined Portfolio - Asset Allocations Over Time')
        axes[0].set_ylabel('Allocation %')
        axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        # Cash allocation
        if cash_cols:
            combined_portfolio[cash_cols].sum(axis=1).plot(ax=axes[1], linewidth=2, color='green')
            axes[1].set_title('Combined Portfolio - Total Cash Allocation')
            axes[1].set_ylabel('Cash %')
            axes[1].grid(True, alpha=0.3)
            
            # Add threshold lines
            axes[1].axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='50% Threshold')
            axes[1].axhline(y=1.0, color='red', linestyle='-', alpha=0.7, label='100% Threshold')
            axes[1].legend()
        
        plt.tight_layout()
        plt.show()
        
        # Portfolio concentration analysis
        print("\nPortfolio Concentration Analysis:")
        
        # Calculate concentration metrics
        herfindahl = (combined_portfolio[asset_cols] ** 2).sum(axis=1)
        max_allocation = combined_portfolio[asset_cols].max(axis=1)
        top3_sum = combined_portfolio[asset_cols].apply(lambda x: x.nlargest(3).sum(), axis=1)
        
        print(f"Average Herfindahl Index: {herfindahl.mean():.3f}")
        print(f"Average maximum allocation: {max_allocation.mean():.1%}")
        print(f"Average top 3 allocation: {top3_sum.mean():.1%}")
        
        # Concentration over time
        fig, ax = plt.subplots(figsize=(15, 6))
        herfindahl.plot(ax=ax, linewidth=1, color='purple')
        ax.set_title('Portfolio Concentration (Herfindahl Index)')
        ax.set_ylabel('Herfindahl Index')
        ax.grid(True, alpha=0.3)
        plt.show()
    
    # Cash allocation analysis
    if cash_cols:
        cash_allocation = combined_portfolio[cash_cols].sum(axis=1)
        print("\nCash Allocation Analysis:")
        print(f"Average cash: {cash_allocation.mean():.1%}")
        print(f"Maximum cash: {cash_allocation.max():.1%}")
        print(f"Minimum cash: {cash_allocation.min():.1%}")
        print(f"Cash volatility: {cash_allocation.std():.1%}")
        
        # Cash allocation distribution
        plt.figure(figsize=(10, 6))
        cash_allocation.hist(bins=50, alpha=0.7)
        plt.title('Cash Allocation Distribution')
        plt.xlabel('Cash %')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # High cash periods analysis
        high_cash_periods = cash_allocation[cash_allocation > 0.5]
        print(f"\nHigh cash periods (>50%): {len(high_cash_periods)} observations")
        if len(high_cash_periods) > 0:
            print(f"Average cash in high periods: {high_cash_periods.mean():.1%}")
            
            # Show some examples
            print("\nExample high cash periods:")
            for date in high_cash_periods.head(5).index:
                cash_pct = cash_allocation.loc[date]
                print(f"  {date.strftime('%Y-%m-%d')}: {cash_pct:.1%} cash")
    
else:
    print("Combined portfolio data not available")

## 4. Turnover and Transaction Analysis

In [ ]:
# Analyze portfolio turnover and transactions
if 'combined_portfolio' in portfolio_data:
    combined_portfolio = portfolio_data['combined_portfolio']
    print("=== TURNOVER AND TRANSACTION ANALYSIS ===\n")
    
    # Calculate portfolio turnover
    asset_cols = [col for col in combined_portfolio.columns if not col.startswith('Cash') and col != 'Total_Value']
    
    if asset_cols:
        # Daily changes in allocations
        allocation_changes = combined_portfolio[asset_cols].diff().abs()
        
        # Turnover metrics
        daily_turnover = allocation_changes.sum(axis=1)
        monthly_turnover = daily_turnover.resample('M').sum()
        annual_turnover = daily_turnover.resample('Y').sum()
        
        print("Portfolio Turnover Analysis:")
        print(f"Average daily turnover: {daily_turnover.mean():.2%}")
        print(f"Average monthly turnover: {monthly_turnover.mean():.2%}")
        print(f"Average annual turnover: {annual_turnover.mean():.2%}")
        print(f"Maximum daily turnover: {daily_turnover.max():.2%}")
        
        # Turnover distribution
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 2, 1)
        daily_turnover.hist(bins=50, alpha=0.7)
        plt.title('Daily Turnover Distribution')
        plt.xlabel('Turnover %')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 2)
        monthly_turnover.hist(bins=20, alpha=0.7)
        plt.title('Monthly Turnover Distribution')
        plt.xlabel('Turnover %')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 3)
        annual_turnover.plot(kind='bar')
        plt.title('Annual Turnover')
        plt.xlabel('Year')
        plt.ylabel('Turnover %')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 4)
        daily_turnover.plot(linewidth=1)
        plt.title('Daily Turnover Over Time')
        plt.xlabel('Date')
        plt.ylabel('Turnover %')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Transaction cost impact
        print("\nTransaction Cost Impact:")
        
        # Estimate transaction costs
        transaction_cost_bps = config['costs']['trading_cost_bps']
        annual_cost_impact = annual_turnover * transaction_cost_bps / 10000
        
        print(f"Trading cost assumption: {transaction_cost_bps} bps")
        print(f"Average annual cost impact: {annual_cost_impact.mean():.2%}")
        print(f"Maximum annual cost impact: {annual_cost_impact.max():.2%}")
        
        # Turnover by asset
        asset_turnover = allocation_changes.mean() * 252  # Annualized
        print("\nAsset Turnover (Annualized):")
        for asset, turnover in asset_turnover.sort_values(ascending=False).head(10).items():
            print(f"  {asset}: {turnover:.1%}")
    
else:
    print("Portfolio data not available for turnover analysis")

## 5. Risk Management Analysis

In [ ]:
# Analyze risk management effectiveness
if 'combined_portfolio' in portfolio_data and 'risk_management' in portfolio_data:
    combined_portfolio = portfolio_data['combined_portfolio']
    risk_data = portfolio_data['risk_management']
    print("=== RISK MANAGEMENT ANALYSIS ===\n")
    
    # Risk signal effectiveness
    if 'stop_loss_signals' in risk_data:
        stop_loss_signals = risk_data['stop_loss_signals']
        print("Stop Loss Signal Analysis:")
        
        for asset, signals in stop_loss_signals.items():
            if not signals.empty:
                signal_freq = signals.mean()
                print(f"  {asset}: {signal_freq:.1%} stop loss triggers")
        
        # Visualize stop loss triggers
        if stop_loss_signals:
            fig, ax = plt.subplots(figsize=(15, 6))
            
            for asset, signals in list(stop_loss_signals.items())[:5]:  # First 5 assets
                signals.astype(int).plot(ax=ax, linewidth=1, alpha=0.7, label=asset)
            
            ax.set_title('Stop Loss Triggers Over Time')
            ax.set_ylabel('Stop Loss Trigger (1=Yes, 0=No)')
            ax.legend()
            ax.grid(True, alpha=0.3)
            plt.show()
    
    # Risk-adjusted allocation analysis
    print("\nRisk-Adjusted Allocation Analysis:")
    
    # Calculate portfolio risk metrics
    asset_cols = [col for col in combined_portfolio.columns if not col.startswith('Cash') and col != 'Total_Value']
    
    if asset_cols and 'risky_assets' in all_data:
        risky_data = all_data['risky_assets']
        
        # Calculate asset volatilities
        asset_volatilities = {}
        for asset in config['assets']['risky']:
            ticker = asset['ticker']
            if ticker in risky_data.columns.levels[0]:
                returns = risky_data[ticker]['Adj Close'].pct_change().dropna()
                if len(returns) > 30:
                    vol = returns.std() * np.sqrt(252)  # Annualized
                    asset_volatilities[ticker] = vol
        
        if asset_volatilities:
            # Portfolio volatility over time
            portfolio_volatility = []
            
            for date in combined_portfolio.index:
                daily_vol = 0
                for asset, alloc in combined_portfolio.loc[date, asset_cols].items():
                    if asset in asset_volatilities and alloc > 0:
                        daily_vol += (alloc * asset_volatilities[asset]) ** 2
                
                portfolio_volatility.append(np.sqrt(daily_vol))
            
            portfolio_vol_series = pd.Series(portfolio_volatility, index=combined_portfolio.index)
            
            print(f"Average portfolio volatility: {portfolio_vol_series.mean():.1%}")
            print(f"Maximum portfolio volatility: {portfolio_vol_series.max():.1%}")
            print(f"Minimum portfolio volatility: {portfolio_vol_series.min():.1%}")
            
            # Plot portfolio volatility
            plt.figure(figsize=(15, 6))
            portfolio_vol_series.plot(linewidth=1, color='red')
            plt.title('Portfolio Volatility Over Time')
            plt.ylabel('Annualized Volatility')
            plt.grid(True, alpha=0.3)
            plt.show()
            
            # Risk reduction during stress periods
            if 'market_health' in signals_dict:
                market_health = signals_dict['market_health']
                if 'Market_Stress_Signal' in market_health.columns:
                    stress_signal = market_health['Market_Stress_Signal']
                    
                    stress_vol = portfolio_vol_series[stress_signal == True].mean()
                    normal_vol = portfolio_vol_series[stress_signal == False].mean()
                    
                    print(f"\nVolatility in stress periods: {stress_vol:.1%}")
                    print(f"Volatility in normal periods: {normal_vol:.1%}")
                    print(f"Risk reduction effectiveness: {(normal_vol - stress_vol) / normal_vol:.1%}")
    
else:
    print("Risk management data not available")

## 6. Portfolio Construction Validation

In [ ]:
# Validate portfolio construction logic
print("=== PORTFOLIO CONSTRUCTION VALIDATION ===\n")

validation_results = {
    'Check': [],
    'Status': [],
    'Details': []
}

# Check 1: Allocation sums to 100%
if 'combined_portfolio' in portfolio_data:
    combined_portfolio = portfolio_data['combined_portfolio']
    asset_cols = [col for col in combined_portfolio.columns if not col.startswith('Cash') and col != 'Total_Value']
    cash_cols = [col for col in combined_portfolio.columns if col.startswith('Cash')]
    
    total_allocation = combined_portfolio[asset_cols + cash_cols].sum(axis=1)
    allocation_error = (total_allocation - 1.0).abs().max()
    
    validation_results['Check'].append('Allocation Sums to 100%')
    if allocation_error < 0.001:
        validation_results['Status'].append('✅ PASS')
        validation_results['Details'].append(f'Max error: {allocation_error:.6f}')
    else:
        validation_results['Status'].append('❌ FAIL')
        validation_results['Details'].append(f'Max error: {allocation_error:.6f}')

# Check 2: No negative allocations
min_allocation = combined_portfolio[asset_cols + cash_cols].min().min()
validation_results['Check'].append('No Negative Allocations')
if min_allocation >= 0:
    validation_results['Status'].append('✅ PASS')
    validation_results['Details'].append(f'Min allocation: {min_allocation:.6f}')
else:
    validation_results['Status'].append('❌ FAIL')
    validation_results['Details'].append(f'Min allocation: {min_allocation:.6f}')

# Check 3: Tranche system working
if 'tranche_positions' in portfolio_data:
    tranche_positions = portfolio_data['tranche_positions']
    tranche_count = len(tranche_positions)
    expected_tranches = config['portfolio']['tranches']['count']
    
    validation_results['Check'].append('Tranche System')
    if tranche_count == expected_tranches:
        validation_results['Status'].append('✅ PASS')
        validation_results['Details'].append(f'{tranche_count} tranches active')
    else:
        validation_results['Status'].append('❌ FAIL')
        validation_results['Details'].append(f'Expected {expected_tranches}, got {tranche_count}')

# Check 4: Rebalancing frequency
if 'combined_portfolio' in portfolio_data:
    # Calculate rebalancing events
    asset_cols = [col for col in combined_portfolio.columns if not col.startswith('Cash') and col != 'Total_Value']
    changes = combined_portfolio[asset_cols].diff().abs().sum(axis=1)
    rebalance_events = (changes > 0.01).sum()  # 1% threshold
    total_days = len(combined_portfolio)
    rebalance_freq = rebalance_events / total_days * 252  # Annualized
    
    validation_results['Check'].append('Rebalancing Frequency')
    expected_freq = 4  # Quarterly
    if abs(rebalance_freq - expected_freq) < 1:
        validation_results['Status'].append('✅ PASS')
        validation_results['Details'].append(f'{rebalance_freq:.1f} times per year')
    else:
        validation_results['Status'].append('⚠️ WARNING')
        validation_results['Details'].append(f'Expected ~{expected_freq}, got {rebalance_freq:.1f}')

# Check 5: Risk management active
if 'risk_management' in portfolio_data:
    risk_data = portfolio_data['risk_management']
    risk_active = len(risk_data) > 0
    
    validation_results['Check'].append('Risk Management')
    if risk_active:
        validation_results['Status'].append('✅ PASS')
        validation_results['Details'].append('Risk signals active')
    else:
        validation_results['Status'].append('❌ FAIL')
        validation_results['Details'].append('No risk signals found')

# Display validation results
validation_df = pd.DataFrame(validation_results)
print(validation_df.to_string(index=False))

print("\nValidation Summary:")
passed = validation_results['Status'].count('✅ PASS')
warnings = validation_results['Status'].count('⚠️ WARNING')
failed = validation_results['Status'].count('❌ FAIL')
total = len(validation_results['Status'])

print(f"Passed: {passed}/{total} ({passed/total:.0%})")
print(f"Warnings: {warnings}/{total}")
print(f"Failed: {failed}/{total}")

if failed == 0 and warnings == 0:
    print("\n🎉 All validation checks passed!")
elif failed == 0:
    print("\n⚠️ Portfolio construction working with minor warnings")
else:
    print("\n❌ Critical issues found - review portfolio construction logic")

print("\n=== PORTFOLIO CONSTRUCTION ANALYSIS COMPLETE ===")
print("\nNext steps:")
print("1. Review validation results and fix any issues")
print("2. Run backtest evaluation (04_backtest_evaluation.ipynb)")
print("3. Compare with benchmark portfolios")